In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/int'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## First impressionts of netflix dataframe:

In [2]:
netflix_titles = pd.read_csv("../input/netflix-shows/netflix_titles.csv", index_col="show_id")
netflix_titles.head(10)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...


In [3]:
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, s1 to s8807
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8807 non-null   object
 1   title         8807 non-null   object
 2   director      6173 non-null   object
 3   cast          7982 non-null   object
 4   country       7976 non-null   object
 5   date_added    8797 non-null   object
 6   release_year  8807 non-null   int64 
 7   rating        8803 non-null   object
 8   duration      8804 non-null   object
 9   listed_in     8807 non-null   object
 10  description   8807 non-null   object
dtypes: int64(1), object(10)
memory usage: 825.7+ KB


In [4]:
netflix_titles.isnull().sum()

type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [5]:
netflix_titles.nunique()

type               2
title           8807
director        4528
cast            7692
country          748
date_added      1767
release_year      74
rating            17
duration         220
listed_in        514
description     8775
dtype: int64

In [6]:
netflix_titles.rating.value_counts()

TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: rating, dtype: int64

Podemos observar que as colunas "director", "cast" e "country" possuem muitos valores nulos (NaN).

Ainda, a coluna "rating" na verdade está categorizando a recomendação de idade mínima para cada série ou filme, não havendo, então, de fato uma coluna com as notas avaliadas aos filmes.

Felizmente o site do IMDB fornece dataframes com as informações necessárias - apenas para fins educacionais, pois para fins profissionais deve ser solicitada permissão e chave de acesso de API.

## Including datasets from IMDB

In [7]:
imdb_titles_akas = pd.read_csv("../input/imdb-educational-proposes/data.tsv", sep="\t", index_col="titleId")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
imdb_titles_akas.head(5)

,ordering,title,region,language,types,attributes,isOriginalTitle
titleId,,,,,,,
tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
tt0000001,2,Carmencita,DE,\N,\N,literal title,0
tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [9]:
imdb_titles_basic = pd.read_csv("../input/titlebasicsimdbeducationalproposes/data.tsv", sep="\t", index_col="tconst")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
imdb_titles_basic.head(20)

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tconst,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000009,short,Miss Jerry,Miss Jerry,0,1894,\N,40,"Romance,Short"


## imdb_titles_basic.info()

In [11]:
imdb_titles_basic[["isAdult","startYear","runtimeMinutes"]] = imdb_titles_basic[["isAdult","startYear","runtimeMinutes"]].replace(regex="\D", value=0).astype(int, errors="raise")

In [12]:
imdb_titles_basic.dtypes

titleType         object
primaryTitle      object
originalTitle     object
isAdult            int64
startYear          int64
endYear           object
runtimeMinutes     int64
genres            object
dtype: object

In [13]:
ratings_imdb = pd.read_csv("../input/ratingimdbeducproposesonly/data.tsv", sep="\t", index_col="tconst")

In [14]:
ratings_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1238776 entries, tt0000001 to tt9916778
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   averageRating  1238776 non-null  float64
 1   numVotes       1238776 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 28.4+ MB


In [15]:
ratings_imdb.head(5)

,averageRating,numVotes
tconst,,
tt0000001,5.7,1874
tt0000002,5.9,248
tt0000003,6.5,1647
tt0000004,5.8,160
tt0000005,6.2,2475


In [16]:
crew_imdb = pd.read_csv("../input/crewimdbeducationalproposes/data.tsv", sep="\t", index_col="tconst")

In [17]:
crew_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8880201 entries, tt0000001 to tt9916880
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   directors  object
 1   writers    object
dtypes: object(2)
memory usage: 203.3+ MB


In [18]:
crew_imdb.head(5)

,directors,writers
tconst,,
tt0000001,nm0005690,\N
tt0000002,nm0721526,\N
tt0000003,nm0721526,\N
tt0000004,nm0721526,\N
tt0000005,nm0005690,\N


In [19]:
crew_imdb.writers.value_counts()
# |N e aprox 48% do valor total de entradas

\N                                                             4333204
nm6352729                                                        12091
nm0934807                                                        11864
nm0914844                                                        11638
nm1271987,nm1667629,nm4390665,nm3089072,nm1956179,nm1335533       9924
                                                                ...   
nm0397574,nm0675310                                                  1
nm11649602,nm3240844,nm3699921                                       1
nm1358790,nm0274160,nm0600694,nm0661312                              1
nm0397574,nm0539315                                                  1
nm6951431                                                            1
Name: writers, Length: 1179659, dtype: int64

In [20]:
crew_imdb.replace("\\N", np.nan, inplace=True)

In [21]:
names = pd.read_csv("../input/namesbasicsimdbeducationalproposes/data.tsv", sep="\t", index_col="nconst")

In [22]:
names.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11586846 entries, nm0000001 to nm9993719
Data columns (total 5 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   primaryName        object
 1   birthYear          object
 2   deathYear          object
 3   primaryProfession  object
 4   knownForTitles     object
dtypes: object(5)
memory usage: 530.4+ MB


In [23]:
names.head(5)

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0071877,tt0038355,tt0117057"
nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0049189,tt0056404,tt0054452"
nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0077975,tt0078723"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0060827,tt0083922,tt0050986"


In [24]:
names.replace("\\N", np.nan, inplace=True)

In [25]:
movie = imdb_titles_akas[imdb_titles_akas['title'] == "9to5: The Story of a Movement"]
movie

,ordering,title,region,language,types,attributes,isOriginalTitle
titleId,,,,,,,
tt11753730,1,9to5: The Story of a Movement,GB,\N,imdbDisplay,\N,0
tt11753730,2,9to5: The Story of a Movement,\N,\N,original,\N,1


In [26]:
test = ratings_imdb.iloc[540641]#["averageRating"] # returns 7.4 float
test

averageRating      7.4
numVotes         228.0
Name: tt11753730, dtype: float64

In [27]:
imdb_titles_basic[imdb_titles_basic.isna().any(axis=1)]

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tconst,,,,,,,,
tt10233364,tvEpisode,Rolling in the Deep Dish\tRolling in the Deep ...,0,2019,0,\N,0,NaN
tt10970874,tvEpisode,Die Bauhaus-Stadt Tel Aviv - Vorbild für die M...,0,2019,0,\N,0,NaN
tt11670006,tvEpisode,...ein angenehmer Unbequemer...\t...ein angene...,0,1981,0,\N,0,NaN
tt11868642,tvEpisode,GGN Heavyweight Championship Lungs With Mike T...,0,2020,0,\N,0,NaN
tt12149332,tvEpisode,Jeopardy! College Championship Semifinal Game ...,0,2020,0,\N,0,NaN
tt12415330,tvEpisode,Anthony Davis High Brow Tank\tAnthony Davis Hi...,0,2017,0,\N,0,NaN
tt13704268,tvEpisode,Bay of the Triffids/Doctor of Doom\tBay of the...,0,0,0,\N,0,NaN
tt14510930,tvEpisode,NaN,NaN,0,0,\N,0,\N
tt15700278,tvEpisode,NaN,NaN,0,2021,\N,0,Talk-Show


In [28]:
test = ratings_imdb.iloc[540641]["averageRating"]
print(test)
print(type(test))

7.4
<class 'numpy.float64'>


In [29]:
print(f"""
netflix_titles : {netflix_titles.columns}
\nimdb_titles_akas : {imdb_titles_akas.columns}
\nimdb_titles_basic : {imdb_titles_basic.columns}
\nnames : {names.columns}
\ncrew_imdb : {crew_imdb.columns}
\nratings_imdb : {ratings_imdb.columns}
""")


netflix_titles : Index(['type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

imdb_titles_akas : Index(['ordering', 'title', 'region', 'language', 'types', 'attributes',
       'isOriginalTitle'],
      dtype='object')

imdb_titles_basic : Index(['titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

names : Index(['primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

crew_imdb : Index(['directors', 'writers'], dtype='object')

ratings_imdb : Index(['averageRating', 'numVotes'], dtype='object')



In [30]:
for idx, line in netflix_titles.head(500).iterrows():
    if line.type == "Movie": # or "Tv Show":
        netflix_title = line.title
        netflix_director = line.director
        netflix_cast = line.cast
        netflix_country = line.country
        
        if netflix_director is "" or netflix_director is np.nan:
            print(f"netflix_title: {netflix_title}\nnetflix_director: {netflix_director}\nnetflix_cast: {netflix_cast}\n\n ")
        
            print("\n----> NaN founded in NETFLIX DIRECTOR!! <----")
            imdb_id_movie = imdb_titles_akas[imdb_titles_akas['title'] == netflix_title].index.values
            crew = crew_imdb.loc[list(imdb_id_movie)].drop_duplicates() # returns DF
            names_idx = [ item for item in crew.directors.values if item is not np.nan ]
            
            for nconst in names_idx:
                if "," not in nconst:
                    name = names.loc[nconst].primaryName # return series
                    print(f"IMDB director: {name}")
                    print("type(name): ", type(name))
                    print(">>netflix_titles.loc[idx][\"director\"]: ", netflix_titles.loc[idx]['director'])
#                     netflix_titles.loc[idx]["director"] = name
#                     netflix_titles[idx]["director"] = name
                    netflix_titles.at[idx,'director'] = name
                    print(">> NEW netflix_titles.loc[idx][\"director\"]: ", netflix_titles.loc[idx]['director'])

                else:
                    inner_nconst = nconst.split(",")
                    director_list = []
                    for n in inner_nconst:
                        name = names.loc[n].primaryName
                        director_list.append(name)
                    print(f"IMDB directorS: {director_list}")
                    netflix_titles.at[idx,'director'] = director_list
                    print(">> NEW netflix_titles.loc[idx][\"director\"]: ", netflix_titles.loc[idx]['director'])
#             print( f"new director line: { line } " )

#         if netflix_cast is np.nan:
#             print("----> NaN founded in NETFLIX CAST !! <----")   

#         if netflix_country is np.nan:
#             print("----> NaN founded in NETFLIX COUNTRY !! ")
        

netflix_title: 9to5: The Story of a Movement
netflix_director: nan
netflix_cast: nan

 

----> NaN founded in NETFLIX DIRECTOR!! <----
IMDB directorS: ['Julia Reichert', 'Steven Bognar']
>> NEW netflix_titles.loc[idx]["director"]:  ['Julia Reichert', 'Steven Bognar']
netflix_title: Bridgerton - The Afterparty
netflix_director: nan
netflix_cast: David Spade, London Hughes, Fortune Feimster

 

----> NaN founded in NETFLIX DIRECTOR!! <----
netflix_title: Last Summer
netflix_director: nan
netflix_cast: Fatih Şahin, Ece Çeşmioğlu, Halit Özgür Sarı, Aslıhan Malbora, Eray Ertüren, Eren Ören, Ozan Kaya Oktu, Talha Öztürk, Merve Nur Bengi, Sümeyye Aydoğan

 

----> NaN founded in NETFLIX DIRECTOR!! <----
IMDB director: Frank Perry
type(name):  <class 'str'>
>>netflix_titles.loc[idx]["director"]:  nan
>> NEW netflix_titles.loc[idx]["director"]:  Frank Perry
IMDB directorS: ['Robert Guédiguian', 'Frank Le Wita']
>> NEW netflix_titles.loc[idx]["director"]:  ['Robert Guédiguian', 'Frank Le Wita']


In [31]:
for idx, line in netflix_titles.head(500).iterrows():
    if line.type == "Tv Show":
        netflix_title = line.title
        netflix_director = line.director
        netflix_cast = line.cast
        netflix_country = line.country
        
        if netflix_director is "" or netflix_director is np.nan:
            print(f"netflix_title: {netflix_title}\nnetflix_director: {netflix_director}\nnetflix_cast: {netflix_cast}\n\n ")
        